In [2]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback, sampling

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm

/home/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Potts but with integers

In [9]:
class Potts():
    def __init__(
        self,
        J: torch.Tensor, # LxqxLxq tensor
        h: torch.Tensor, # Lxq tensor
    ):
        super().__init__()
        sz_h = h.size()
        sz_J = J.size()
        if len(sz_h) != 2:
            raise ValueError(f"Expected tensor with 2 dimensions, got {len(sz_h)}.")
        if len(sz_J) != 4:
            raise ValueError(f"Expected tensor with 2 dimensions, got {len(sz_J)}.")
        if not (sz_J[0:2] == sz_J[2:4] == sz_h):
            raise ValueError("Wrong tensor dimensions")
        
        self.h = torch.nn.Parameter(h)

        L, q = sz_h
        mask = torch.ones(L, q, L, q, device=J.device)
        # set the (i,i) blocks to zero
        mask[torch.arange(L), :, torch.arange(L), :] = 0
        self.J = torch.nn.Parameter(J)
        self.mask = mask

    def get_n_states(self):
        return self.h.size(1)

    def get_sequence_length(self):
        return self.h.size(0)

    def compute_energy(
        self,
        x: torch.Tensor
    ):
        L = self.get_sequence_length()
        q = self.get_n_states()
        # the -1 accounts for possible batch index along dimension 0
        x_flat = x.view(-1, L * q)
        bias_flat = self.h.view(L * q)
        couplings_flat = (self.J * self.mask).reshape(L * q, L * q)
        bias_term = x_flat @ bias_flat
        coupling_term = torch.sum(x_flat * (x_flat @ couplings_flat), dim=1)
        return - bias_term - 0.5 * coupling_term

In [10]:
L = 4
q = 3
h = torch.randn(L, q)
J_ = torch.randn(L*q, L*q)
J_ = J_ + J_.t()
J = J_.reshape(L, q, L, q)

In [11]:
x = torch.randint(q, (L,))
x_oh = one_hot(x)

In [24]:
potts = energy_models.Potts(J,h)
potts.compute_energy(x_oh).item()

-0.7279629111289978

In [15]:
(x_oh * h).sum()

tensor(0.0735)

In [21]:
h[x]

tensor([[ 0.1808,  0.7356,  0.1404],
        [ 1.1095,  0.2422, -0.3267],
        [-1.9800, -1.6846, -0.3730],
        [-1.9800, -1.6846, -0.3730]])

In [31]:
mask = potts.mask
def en_potts(J, h, x):
    e_fields = - h.gather(1, x.unsqueeze(1)).sum()
    i_idx, j_idx = torch.arange(L, device=J.device)[:, None], torch.arange(L, device=J.device)[None, :]
    e_couplings = - 0.5 * (J * mask)[i_idx, x[:, None], j_idx, x[None, :]].sum()
    return e_fields + e_couplings

In [32]:
%timeit en_potts(J, h, x)

17.9 μs ± 18.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [33]:
%timeit potts.compute_energy(x_oh).item()

14.6 μs ± 17.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
